In [8]:
from core.db import get_engine
from core.schema_helpers import get_column
from core.libs import pd
from sqlalchemy import text

def crear_cat_inventory_para_todas():
    engine = get_engine()
    # Buscar todas las tablas que empiecen con inventory_
    tables = pd.read_sql("""
        SELECT table_name
        FROM information_schema.tables
        WHERE table_schema = 'public'
        AND table_name LIKE 'inventory_%';
    """, engine)["table_name"].tolist()

    for table in tables:
        print(f"📦 Procesando {table}...")

        df = pd.read_sql(f'SELECT * FROM public."{table}"', engine)
        if df.empty:
            print(f"⚠️  {table} está vacía, se omite.")
            continue

        # Buscar columnas usando schema_helpers
        try:
            contract_col = get_column("contractcode", df)
            farmer_col   = get_column("farmername", df)
            cruise_col   = get_column("cruisedate", df)
        except KeyError as e:
            print(f"⛔ Error columnas faltantes en {table}: {e}")
            continue

        cat_df = (
            df[[contract_col, farmer_col, cruise_col]]
            .drop_duplicates(subset=[contract_col])
            .reset_index(drop=True)
        )

        # Nombre destino tipo cat_inventory_mx_2025
        suffix = table.replace("inventory_", "")
        dest = f"cat_inventory_{suffix}"

        # Guardar en SQL
        cat_df.to_sql(dest, engine, index=False, if_exists="replace")
        print(f"✅ cat_inventory generado: {dest} ({len(cat_df)} contratos)")

if __name__ == "__main__":
    crear_cat_inventory_para_todas()


💻 Conectado a la base de datos helloworldtree


TypeError: sqlalchemy.cyextension.immutabledict.immutabledict is not a sequence